# Evaluating an N-Gram Language Model



In [1]:
from n_gram import NGramLM

BOS = '<BOS>'
EOS = '<EOS>'
OOV = '<OOV>'

# Load pre-built n-gram languae models
model_unigram = NGramLM('arthur-conan-doyle.tok.train.n1.pkl')
model_bigram = NGramLM('arthur-conan-doyle.tok.train.n2.pkl')
model_trigram = NGramLM('arthur-conan-doyle.tok.train.n3.pkl')
model_4gram = NGramLM('arthur-conan-doyle.tok.train.n4.pkl')
model_5gram = NGramLM('arthur-conan-doyle.tok.train.n5.pkl')

In [30]:
NGramLM.from_file("bu")

bu


In [31]:
BOS in whatever['V'], EOS in whatever['V'], OOV in whatever['V']

(True, True, True)

In [32]:
BOS in whatever['model'], EOS in whatever['model'], OOV in whatever['model']

(False, False, False)

In [33]:
(BOS,) in whatever_2['model'], (EOS,) in whatever_2['model'], (OOV,) in whatever_2['model']

(True, False, False)

In [34]:
whatever["model"].keys()

dict_keys([()])

In [35]:
whatever_2["model"].keys()

dict_keys([('<BOS>',), ('The',), ('Adventure',), ('of',), ('Wisteria',), ('Lodge',), ('Singular',), ('Experience',), ('Mr.',), ('John',), ('Scott',), ('Eccles',), ('Tiger',), ('San',), ('Pedro',), ('1',), ('.',), ('I',), ('find',), ('it',), ('recorded',), ('in',), ('my',), ('notebook',), ('that',), ('was',), ('a',), ('bleak',), ('and',), ('windy',), ('day',), ('towards',), ('the',), ('end',), ('March',), ('year',), ('1892',), ('Holmes',), ('had',), ('received',), ('telegram',), ('while',), ('we',), ('sat',), ('at',), ('our',), ('lunch',), (',',), ('he',), ('scribbled',), ('reply',), ('He',), ('made',), ('no',), ('remark',), ('but',), ('matter',), ('remained',), ('his',), ('thoughts',), ('for',), ('stood',), ('front',), ('fire',), ('afterwards',), ('with',), ('thoughtful',), ('face',), ('smoking',), ('pipe',), ('casting',), ('an',), ('occasional',), ('glance',), ('message',), ('Suddenly',), ('turned',), ('upon',), ('me',), ('mischievous',), ('twinkle',), ('eyes',), ('"',), ('suppose',),

In [36]:
whatever_2['V'] == whatever['V'], type(whatever['V'])

(True, list)

In [37]:
whatever_2['V'][:10]

['<BOS>', '<EOS>', '<OOV>', '!', '"', '&', "'", "'S", "'d", "'em"]

In [38]:
import re

to_process = "arthur-conan-doyle.tok.train.n2.pkl"
n_value = re.search(r'n(\d+)', to_process).group(1)
print(n_value)

2


In [39]:
whatever_2['n'], whatever['n']

(2, 1)

In [40]:
from collections import Counter
from n_gram import *


def load_ngrams(path: str) -> dict:
    """
    Load n-grams from a file.

    Args:
        path (str): Path to the file.

    Returns:
        dict: A dictionary containing the loaded n-grams.
    """
    try:
        with open(path, 'rb') as fin:
            data = pickle.load(fin)
    except FileNotFoundError:
        import re
        n_value = int(re.search(r'n(\d+)', path).group(1))
        data = {'n': n_value}
        with open("arthur-conan-doyle.tok.train.txt", 'rb') as corpus:
            full_text = [line.decode().split() for line in corpus.readlines()]
        data['model'] = count_ngrams(full_text, n=data['n'])
        data['V'] = [BOS, EOS, OOV] + [word for line in full_text for word in line]
        pickle.dump(data, open(path, "wb"))
    return data


local_model = load_ngrams("aaa.n2.pkl")

In [41]:
set(local_model['V']) == set(whatever['V']), set(local_model['V']) == set(whatever_2['V'])

(True, True)

In [42]:
local_model['model']

{('<BOS>',): {'The': 293,
  '1': 1,
  'I': 161,
  '"': 7790,
  'He': 97,
  'A': 55,
  'Holmes': 222,
  'Our': 48,
  'But': 27,
  'Out': 1,
  'Sherlock': 47,
  'Mr.': 24,
  'My': 39,
  'We': 66,
  'An': 8,
  'Lord': 7,
  'It': 136,
  '2': 1,
  'In': 19,
  'Inspector': 3,
  'Occasionally': 1,
  'THE': 27,
  'Baynes': 2,
  'At': 19,
  'Mycroft': 2,
  'See': 1,
  'His': 13,
  'Yet': 1,
  'Surely': 1,
  'There': 36,
  'Am': 1,
  'Caulfield': 1,
  'One': 7,
  'And': 13,
  'Colonel': 8,
  'Then': 1,
  'On': 8,
  'These': 4,
  'Mortimer': 2,
  'With': 13,
  'Little': 2,
  'They': 1,
  'For': 14,
  'Sterndale': 4,
  'Dr.': 6,
  'PART': 2,
  'So': 10,
  'When': 23,
  'Again': 11,
  'As': 28,
  'She': 25,
  'Half': 3,
  'Two': 3,
  'Only': 3,
  'Already': 1,
  'To': 6,
  'Through': 2,
  'Three': 3,
  'Her': 2,
  'Peters': 1,
  'Finally': 1,
  'Five': 2,
  'Mrs.': 3,
  'Yes': 1,
  'Poor': 1,
  'Never': 3,
  'This': 3,
  'Lower': 1,
  'From': 9,
  'Von': 14,
  'Will': 1,
  'Both': 2,
  '+': 2,
  'S

In [43]:
whatever_2['model'].keys() == local_model['model'].keys()

True

In [44]:
whatever_2['model'].values() == local_model['model'].values()

False

In [45]:
for key in whatever_2['model'].keys():
    val_1 = whatever_2['model'][key]
    val_2 = local_model['model'][key]
    if val_1 != val_2:
        print(f"Mismatch for key {key}: {val_1} != {val_2}")
        break

In [46]:
full_text = []
with open('arthur-conan-doyle.tok.train.txt', 'rt') as fin:
    for line in fin:
        full_text.append(list(line.split()))
print(full_text[:10])

local_model = load_ngrams("aaa.n2.txt")
word_counts = Counter(word for line in full_text for word in line)
word_counts["The"]

[['The', 'Adventure', 'of', 'Wisteria', 'Lodge'], ['The', 'Singular', 'Experience', 'of', 'Mr.', 'John', 'Scott', 'Eccles', 'The', 'Tiger', 'of', 'San', 'Pedro'], ['1', '.', 'The', 'Singular', 'Experience', 'of', 'Mr.', 'John', 'Scott', 'Eccles'], ['I', 'find', 'it', 'recorded', 'in', 'my', 'notebook', 'that', 'it', 'was', 'a', 'bleak', 'and', 'windy', 'day', 'towards', 'the', 'end', 'of', 'March', 'in', 'the', 'year', '1892', '.', 'Holmes', 'had', 'received', 'a', 'telegram', 'while', 'we', 'sat', 'at', 'our', 'lunch', ',', 'and', 'he', 'had', 'scribbled', 'a', 'reply', '.', 'He', 'made', 'no', 'remark', ',', 'but', 'the', 'matter', 'remained', 'in', 'his', 'thoughts', ',', 'for', 'he', 'stood', 'in', 'front', 'of', 'the', 'fire', 'afterwards', 'with', 'a', 'thoughtful', 'face', ',', 'smoking', 'his', 'pipe', ',', 'and', 'casting', 'an', 'occasional', 'glance', 'at', 'the', 'message', '.', 'Suddenly', 'he', 'turned', 'upon', 'me', 'with', 'a', 'mischievous', 'twinkle', 'in', 'his', 'e

1614